## Simple Test

Joystick Controller class

In [1]:
import digitalio
from digitalio import DigitalInOut, Direction
from adafruit_rgb_display import st7789
import board

class JoystickController:
    def __init__(self):
        # 버튼 설정
        self.button_U = digitalio.DigitalInOut(board.D17)
        self.button_D = digitalio.DigitalInOut(board.D22)
        self.button_L = digitalio.DigitalInOut(board.D27)
        self.button_R = digitalio.DigitalInOut(board.D23)
        self.button_A = digitalio.DigitalInOut(board.D5) 
        self.button_B = digitalio.DigitalInOut(board.D6)

        self.cs_pin = DigitalInOut(board.CE0)
        self.dc_pin = DigitalInOut(board.D25)
        self.reset_pin = DigitalInOut(board.D24)
        self.BAUDRATE = 24000000
        self.spi = board.SPI()
        self.disp = st7789.ST7789(
                    self.spi,
                    height=240,
                    y_offset=80,
                    rotation=180,
                    cs=self.cs_pin,
                    dc=self.dc_pin,
                    rst=self.reset_pin,
                    baudrate=self.BAUDRATE,
                    )
        # 버튼 입력 방향 설정
        self.button_U.direction = digitalio.Direction.INPUT
        self.button_D.direction = digitalio.Direction.INPUT
        self.button_L.direction = digitalio.Direction.INPUT
        self.button_R.direction = digitalio.Direction.INPUT
        self.button_A.direction = digitalio.Direction.INPUT
        self.button_B.direction = digitalio.Direction.INPUT
        # 버튼 상태
        self.button_states = {
            "up": False,
            "down": False,
            "left": False,
            "right": False,
            "a": False,  # A 버튼 상태
            "b": False   # B 버튼 상태 추가
        }
        # Turn on the Backlight
        self.backlight = DigitalInOut(board.D26)
        self.backlight.switch_to_output()
        self.backlight.value = True

        # Create blank image for drawing.
        # Make sure to create image with mode 'RGB' for color.
        self.width = self.disp.width
        self.height = self.disp.height

    def update(self):
        # 각 버튼의 상태를 업데이트
        self.button_states["up"] = not self.button_U.value
        self.button_states["down"] = not self.button_D.value
        self.button_states["left"] = not self.button_L.value
        self.button_states["right"] = not self.button_R.value
        self.button_states["a"] = not self.button_A.value 
        self.button_states["b"] = not self.button_B.value

    def is_button_pressed(self, button):
        # 특정 버튼이 눌렸는지 확인
        return self.button_states.get(button, False)

    def get_joystick_direction(self):
        # 조이스틱 방향 반환 (버튼 기반)
        direction = []
        if self.is_button_pressed("up"):
            direction.append("up")
        if self.is_button_pressed("down"):
            direction.append("down")
        if self.is_button_pressed("left"):
            direction.append("left")
        if self.is_button_pressed("right"):
            direction.append("right")
        return direction

Player class

In [2]:
from PIL import Image, ImageDraw
import random
import time

sprite_files = {
    "normal": ["/home/kau-esw/Desktop/DancingDino/img/player_idle1.PNG", "/home/kau-esw/Desktop/DancingDino/img/player_idle2.PNG"],
    "left": ["/home/kau-esw/Desktop/DancingDino/img/player_idle1.PNG", "/home/kau-esw/Desktop/DancingDino/img/player_left.PNG"],
    "right": ["/home/kau-esw/Desktop/DancingDino/img/player_idle1.PNG", "/home/kau-esw/Desktop/DancingDino/img/player_right.PNG"],
    "up": ["/home/kau-esw/Desktop/DancingDino/img/player_idle1.PNG", "/home/kau-esw/Desktop/DancingDino/img/player_up.PNG"],
    "down": ["/home/kau-esw/Desktop/DancingDino/img/player_idle1.PNG", "/home/kau-esw/Desktop/DancingDino/img/player_down.PNG"],
    "leftup": ["/home/kau-esw/Desktop/DancingDino/img/player_idle1.PNG", "/home/kau-esw/Desktop/DancingDino/img/player_leftup.PNG"],
    "rightup": ["/home/kau-esw/Desktop/DancingDino/img/player_idle1.PNG", "/home/kau-esw/Desktop/DancingDino/img/player_rightup.PNG"],
}

class Player:
    def __init__(self, sprite_files, disp):
        self.sprites = {pose: [Image.open(file) for file in files] for pose, files in sprite_files.items()}
        self.current_sprite_index = 0
        self.current_pose = 'normal'
        self.disp = disp
        self.last_update_time = time.time()
        self.update_interval = 0.5  # 업데이트 간격
        self.feedback = ""
        self.score = 0
    
    def update_pose(self, pose):
        self.current_pose = pose
        self.current_sprite_index = 0

    def update_animation(self):
        current_time = time.time()
        if current_time - self.last_update_time >= self.update_interval:
            self.last_update_time = current_time
            self.current_sprite_index = (self.current_sprite_index + 1) % len(self.sprites[self.current_pose])

    def update_score(self, feedback):
        self.feedback = feedback
        if feedback == "perfect":
            self.score += 100
        elif feedback == "good":
            self.score += 50
        elif feedback == "miss":
            self.score += 0  # 또는 점수 감점
    
    def render_feedback(self, image_draw, feedback):
        self.feedback = feedback
        # 예: "Perfect", "Good", "Miss" 등의 텍스트 렌더링
        image_draw.text((50, 100), self.feedback, fill=(255, 255, 255))
        self.feedback = ""
    
    def render(self, draw, image):
        # 이미지를 RGBA 모드로 변경
        image = image.convert("RGBA")

        # 각 zone의 높이 설정 (위 zone부터 시작)
        zone_heights = [70, 60, 20]  # 각 zone의 높이

        # 각 zone의 색상 설정 (RGBA 모드)
        zone_colors = [(255, 0, 0, 128),  # 반투명 빨간색
                       (0, 255, 0, 128),  # 반투명 초록색
                       (0, 0, 255, 128)]  # 반투명 파란색

        # zone 렌더링
        current_y = 0
        for i in range(3):
            zone_height = zone_heights[i]
            zone_image = Image.new("RGBA", (self.disp.width, zone_height), zone_colors[i])
            image.alpha_composite(zone_image, (0, current_y))
            current_y += zone_height

        # 다시 RGB 모드로 변환
        image = image.convert("RGB")

        # 피드백 메시지 및 스프라이트 렌더링
        draw = ImageDraw.Draw(image)
        self.render_feedback(draw, self.feedback)
    
        sprite = self.sprites[self.current_pose][self.current_sprite_index]
        image.paste(sprite, (100, 160), sprite)

        return image

Arrow class

In [3]:
arrow_images = {
    "up": Image.open("/home/kau-esw/Desktop/DancingDino/img/arrow_up.PNG"),
    "down": Image.open("/home/kau-esw/Desktop/DancingDino/img/arrow_down.PNG"),
    "left": Image.open("/home/kau-esw/Desktop/DancingDino/img/arrow_left.PNG"),
    "right": Image.open("/home/kau-esw/Desktop/DancingDino/img/arrow_right.PNG"),
    "leftup": Image.open("/home/kau-esw/Desktop/DancingDino/img/arrow_leftup.PNG"),
    "rightup": Image.open("/home/kau-esw/Desktop/DancingDino/img/arrow_rightup.PNG")
}

class Arrow:
    def __init__(self, direction, y_start, screen_width, screen_height, arrow_images, long=False):
        self.direction = direction
        self.x = screen_width // 2
        self.y = y_start
        self.screen_width = screen_width
        self.screen_height = screen_height
        self.long = long
        self.active = True
        self.arrow_image = arrow_images[direction]  # 이미지를 딕셔너리에서 불러옴

    def render(self, draw, image):
        # 화살표 이미지를 현재 위치에 그림
        image_x = self.x - self.arrow_image.width // 2
        image_y = self.y - self.arrow_image.height // 2
        # 화살표 이미지 그리기
        image.paste(self.arrow_image, (image_x, image_y), self.arrow_image)

        if self.long:
            # 긴 화살표의 경우, 모든 화살표에 대해 아래에서 위쪽으로 이어지는 꼬리 추가
            tail_length = 100  # 꼬리의 길이
            tail_width = 10   # 꼬리의 너비

            # 꼬리의 시작점과 끝점
            tail_start = (self.x, self.y)
            tail_end = (self.x, self.y - tail_length)

            # 꼬리 그리기 (노란색 꼬리에 검은색 아웃라인)
            draw.rectangle([tail_start[0] - tail_width // 2, tail_start[1],
                            tail_end[0] + tail_width // 2, tail_end[1]], fill="yellow")
            draw.line([tail_start, tail_end], fill="black", width=tail_width + 2)

    def update(self):
        self.y += 10
        if self.y > self.screen_height:
            self.active = False

Main Game flow

In [ ]:
import time
import random

class Game:
    def __init__(self, joystick, player):
        self.disp = joystick.disp
        self.joystick = joystick
        self.player = player
        self.arrows = []
        self.score = 0
        self.zones = [(0, 70), (70, 130), (130, 150)]  # 점수 존 위치
        self.arrow_types = ["left", "right", "down", "up", "rightup", "leftup"]
        self.arrow_images = arrow_images  # 화살표 이미지 딕셔너리
        self.spawn_interval = 2  # 화살표 생성 간격 (초)
        self.last_spawn_time = time.time()
        self.start_screen = True
        self.game_over = False
        self.timer = 30.0  # 30-second timer

    def start_game(self):
        self.start_screen = False

    def end_game(self):
        self.game_over = True

    def update(self):
        if self.start_screen:
            if self.joystick.is_button_pressed('b'):
                self.start_game()
                print('game start!')
                return
        elif not self.game_over:
            # Game logic here
            self.spawn_arrow()
            self.update_arrows()
            # 사용자 입력에 따라 자세 업데이트
            pose = self.get_charactor_pose()
            self.player.update_pose(pose)
            dir = self.joystick.get_joystick_direction()
            self.player.update_animation()
            # Check for player input and update the score
            feedback = self.check_hit(pose)
            if feedback:
                self.player.update_score(feedback)
            # Update the timer
            self.timer -= 0.2
            if self.timer <= 0:
                self.end_game()

    def get_charactor_pose(self):
        direction = self.joystick.get_joystick_direction()
        if len(direction) == 0:
            return 'normal'
        elif len(direction) == 1:
            return direction[0]
        elif (direction[0] == "left" and direction[1] == "up") or (direction[0] == "up" and direction[1] == "left"):
            return "leftup"
        elif (direction[0] == "right" and direction[1] == "up") or (direction[0] == "up" and direction[1] == "right"):
            return "rightup"

    def spawn_arrow(self):
        current_time = time.time()
        if current_time - self.last_spawn_time >= self.spawn_interval:
            self.last_spawn_time = current_time
            direction = random.choice(self.arrow_types)
            # 새 화살표 객체 생성 시 화살표 이미지 딕셔너리 참조 포함
            new_arrow = Arrow(direction, 0, self.disp.width, self.disp.height, self.arrow_images)
            self.arrows.append(new_arrow)
    
    def update_arrows(self):
        for arrow in self.arrows:
            arrow.update()
            if not arrow.active:
                self.arrows.remove(arrow)
            
    def render(self):
        image = Image.new("RGB", (self.disp.width, self.disp.height))
        draw = ImageDraw.Draw(image)
        if self.start_screen:
            draw.text((10, 10), "Press 'B' to Start", fill=(255, 255, 255))
        elif not self.game_over:
            # Render game elements as before
            image = self.player.render(draw, image)
            for arrow in self.arrows:
                arrow.render(draw, image)
            draw.text((10, 10), f"Score: {self.score}", fill=(255, 255, 255))
            draw.text((10, 40), f"Time Left: {int(self.timer)}", fill=(255, 255, 255))
            self.disp.image(image)
        else:
            # Display game-over screen with final score
            draw.text((10, 10), "Game Over", fill=(255, 255, 255))
            draw.text((10, 40), f"Final Score: {self.score}", fill=(255, 255, 255))

    def check_hit(self, player_input):
        if len(self.arrows) == 0:
            return ""
        arrow = self.arrows[0]
        zone = self.calculate_zone(arrow.y)
        if zone is None:
            arrow.active = False
        if arrow.active and arrow.direction != player_input and player_input != 'normal':
            arrow.active == False
            return "Miss"
        if arrow.active and arrow.direction == player_input:
            arrow.active == False
            if zone == 2:
                self.score += 100
                return "Perfect"
            elif zone == 1:
                self.score += 50
                return "Good"
            else:
                return "Miss"
        return ""
        
    def calculate_zone(self, arrow_y):
        arrow_bottom = arrow_y + 30  # arrow 박스가 zone에 겹쳐지는 지 계산
        for idx, (start, end) in enumerate(self.zones):
            if start <= arrow_bottom <= end:
                return idx
        return None
            
def main():
    joystick = JoystickController()
    player = Player(sprite_files, joystick.disp)
    game1 = Game(joystick, player)

    while True:
        # 조이스틱 입력 업데이트
        if game1.game_over:
            break
        joystick.update()
         # 화살표 생성 및 업데이트
        game1.update()
        # 화면 렌더링
        game1.render()
        time.sleep(0.2)

if __name__ == "__main__":
    main()